In [1]:
import time
import pandas as pd
from index import InvertedIndex
from pympler import asizeof
import pickle

def get_time(func, *args, **kwargs):
    """
    Вывод времени работы функции
    """
    start = time.time()
    res = func(*args, **kwargs)
    end = time.time()
    func_time = round(end - start, 6)
    return res, func_time

def get_object_size(some_object):
    """
    Возвращает размер объекта в килобайтах
    """
    size = asizeof.asizeof(some_object) / 1024
    return size

# Индексирование данных из модуля №2

In [2]:
# загрузка данных и их соединение
msu_df = pd.read_csv('../../module_2/data/msu_september.csv')
spbgu_df = pd.read_csv('../../module_2/data/spbgu_september.csv')

df = pd.concat([msu_df, spbgu_df])
df.sample(1)

,Unnamed: 0,id,from_id,date,text,likes,views,reposts,comments,topic
2,2,23050,-140513133,2024-09-02,ПОЗДРАВЛЯЕМ!! \nАЛЕКСЕЮ ЧАДОВУ 43 ГОДА!! \nАле...,61,800,4,1,СПБГУ


In [3]:
# формируем словарь документов
documents = dict(zip(df['id'], df['text'])) 

In [4]:
# строим индекс
index = InvertedIndex()

for document_id, text in documents.items():
    if type(text) == str:
        index.add_document(document_id, text)

In [5]:
# сохранение индекса в pickle, чтобы не строить заново каждый раз
with open('../data/inverted_index.pkl', 'wb') as f:
    pickle.dump(index, f)   

## Поиск в простом инвертированном индексе

In [6]:
with open('../data/inverted_index.pkl', 'rb') as f:
    index = pickle.load(f)  

print('Размер простого индекса: ', get_object_size(index.get_index()))

Размер простого индекса:  57223.84375


In [7]:
# Поиск
query_spb = "Ректор СПБГУ"
query_msu = "Ректор МГУ"

result_spb, time_of_complete_spb = get_time(index.search, query_spb)
result_msu, time_of_complete_msu = get_time(index.search, query_msu)

print(f'Время поиска <{query_spb}> в простом индексе - {time_of_complete_spb}')
print(f'Время поиска <{query_msu}> в простом индексе - {time_of_complete_msu}')

Время поиска <Ректор СПБГУ> в простом индексе - 0.010204
Время поиска <Ректор МГУ> в простом индексе - 0.002434


In [8]:
print(f"Документы, содержащие <{query_spb}>: {result_spb}")
print(f"Всего их - {len(result_spb)}")

print()

print(f"Документы, содержащие <{query_msu}>: {result_msu}")
print(f"Всего их - {len(result_msu)}")

Документы, содержащие <Ректор СПБГУ>: [3586, 102788, 1542, 5646, 3599, 428688, 3862, 16023, 5144, 3478, 2586, 6938, 15654, 1451, 4912, 946, 2483, 1589, 949, 4791, 1215, 8768, 2368, 61131, 19532, 337, 1235, 1236, 2134, 1238, 473, 8412, 11373, 110, 4856, 763, 11133, 2431]
Всего их - 38

Документы, содержащие <Ректор МГУ>: [2560, 5646, 3599, 2584, 5144, 12312, 2586, 3616, 2596, 261672, 455211, 22066, 24634, 8768, 4164, 46150, 8268, 144974, 4177, 2133, 8280, 19046, 112, 15484, 7808, 1667, 9863, 21128, 18568, 4747, 142, 428688, 145, 2194, 658, 148, 2200, 1689, 4248, 1692, 8860, 24734, 3755, 4791, 2744, 184, 30393, 6843, 91324, 17083, 30394, 13510, 9928, 5321, 37076, 213, 37078, 1238, 48347, 23260, 8924, 8926, 1761, 48355, 1767, 3816, 5865, 27379, 2805, 17653, 4856, 763, 32512, 6919, 5897, 20235, 11545, 2331, 54555, 18206, 16159, 143650, 2339, 807, 20264, 2345, 4912, 1845, 15669, 1856, 15683, 23368, 333, 6479, 6480, 337, 13138, 5462, 130398, 3946, 34172, 102788, 3478, 143255, 34202, 1435, 66

In [9]:
# пример
for doc_id in result_spb:
    print(f"Документ {doc_id}: {index.get_document(doc_id)}")
    break

Документ 3586: Документ в рамках Восточного экономического форума подписали
ректор ЗабГУ Оксана Олеговна Мартыненко и проректор по научной работе СПбГУ
Сергей Владимирович Микушев. 

Санкт-Петербургский государственный университет и
Забайкальский государственный университет займутся разработкой сетевых
образовательных программ для подготовки инженеров. Также вузы будут совместно
готовить специалистов в области международных отношений, экономики и
востоковедения. Партнеры договорились вместе организовывать академические и
научные мероприятия, образовательные программы и курсы.
 “СПбГУ - ведущий университет России,  один из флагманов мировой науки и высшего
образования. Мощь и потенциал СПбГУ охватывает огромный спектр предметных
областей. Соглашение о сотрудничестве ЗабГУ с СПбГУ открывает новые возможности
для развития всех факультетов и кафедр нашего университета. Сегодня мы начинаем
сотрудничество с передовой инженерной школой СПбГУ в области геологии и горного
дела, договорились о с

## Сжатие гамма-кодированием и поиск

In [10]:
with open('../data/inverted_index.pkl', 'rb') as f:
    index = pickle.load(f)   

In [11]:
res, time_of_complete = get_time(index.compress_index, 'gamma')
print('Время сжатия:', time_of_complete)

Время сжатия: 1.00948


In [12]:
print('Размер сжатого гамма-кодированием индекса: ', get_object_size(index.get_index()))

Размер сжатого гамма-кодированием индекса:  16514.0390625


In [13]:
# Поиск
query_spb = "Ректор СПБГУ"
query_msu = "Ректор МГУ"

result_spb, time_of_complete_spb = get_time(index.search, query_spb)
result_msu, time_of_complete_msu = get_time(index.search, query_msu)

print(f'Время поиска <{query_spb}> в сжатом гамма-кодированием индексе - {time_of_complete_spb}')
print(f'Время поиска <{query_msu}> в сжатом гамма-кодированием индексе - {time_of_complete_msu}')

Время поиска <Ректор СПБГУ> в сжатом гамма-кодированием индексе - 0.014707
Время поиска <Ректор МГУ> в сжатом гамма-кодированием индексе - 0.0428


In [14]:
print(f"Документы, содержащие <{query_spb}>: {result_spb}")
print(f"Всего их - {len(result_spb)}")

print()

print(f"Документы, содержащие <{query_msu}>: {result_msu}")
print(f"Всего их - {len(result_msu)}")

Документы, содержащие <Ректор СПБГУ>: [3586, 102788, 1542, 5646, 3599, 428688, 3862, 16023, 5144, 3478, 2586, 6938, 15654, 1451, 4912, 946, 2483, 1589, 949, 4791, 1215, 8768, 2368, 61131, 19532, 337, 1235, 1236, 2134, 1238, 473, 8412, 11373, 110, 4856, 763, 11133, 2431]
Всего их - 38

Документы, содержащие <Ректор МГУ>: [2560, 5646, 3599, 2584, 5144, 12312, 2586, 3616, 2596, 261672, 455211, 22066, 24634, 8768, 4164, 46150, 8268, 144974, 4177, 2133, 8280, 19046, 112, 15484, 7808, 1667, 9863, 21128, 18568, 4747, 142, 428688, 145, 2194, 658, 148, 2200, 1689, 4248, 1692, 8860, 24734, 3755, 4791, 2744, 184, 30393, 6843, 91324, 17083, 30394, 13510, 9928, 5321, 37076, 213, 37078, 1238, 48347, 23260, 8924, 8926, 1761, 48355, 1767, 3816, 5865, 27379, 2805, 17653, 4856, 763, 32512, 6919, 5897, 20235, 11545, 2331, 54555, 18206, 16159, 143650, 2339, 807, 20264, 2345, 4912, 1845, 15669, 1856, 15683, 23368, 333, 6479, 6480, 337, 13138, 5462, 130398, 3946, 34172, 102788, 3478, 143255, 34202, 1435, 66

## Сжатие дельта-кодированием и поиск

In [15]:
with open('../data/inverted_index.pkl', 'rb') as f:
    index = pickle.load(f)   

In [16]:
res, time_of_complete = get_time(index.compress_index, 'delta')
print('Время сжатия:', time_of_complete)

Время сжатия: 2.447611


In [17]:
print('Размер сжатого дельта-кодированием индекса: ', get_object_size(index.get_index()))

Размер сжатого дельта-кодированием индекса:  15570.34375


In [20]:
# Поиск
query_spb = "Ректор СПБГУ"
query_msu = "Ректор МГУ"

result_spb, time_of_complete_spb = get_time(index.search, query_spb)
result_msu, time_of_complete_msu = get_time(index.search, query_msu)

print(f'Время поиска <{query_spb}> в сжатом дельта-кодированием индексе - {time_of_complete_spb}')
print(f'Время поиска <{query_msu}> в сжатом дельта-кодированием индексе - {time_of_complete_msu}')

Время поиска <Ректор СПБГУ> в сжатом дельта-кодированием индексе - 0.02048
Время поиска <Ректор МГУ> в сжатом дельта-кодированием индексе - 0.052921


In [19]:
print(f"Документы, содержащие <{query_spb}>: {result_spb}")
print(f"Всего их - {len(result_spb)}")

print()

print(f"Документы, содержащие <{query_msu}>: {result_msu}")
print(f"Всего их - {len(result_msu)}")

Документы, содержащие <Ректор СПБГУ>: [3586, 102788, 1542, 5646, 3599, 428688, 3862, 16023, 5144, 3478, 2586, 6938, 15654, 1451, 4912, 946, 2483, 1589, 949, 4791, 1215, 8768, 2368, 61131, 19532, 337, 1235, 1236, 2134, 1238, 473, 8412, 11373, 110, 4856, 763, 11133, 2431]
Всего их - 38

Документы, содержащие <Ректор МГУ>: [2560, 5646, 3599, 2584, 5144, 12312, 2586, 3616, 2596, 261672, 455211, 22066, 24634, 8768, 4164, 46150, 8268, 144974, 4177, 2133, 8280, 19046, 112, 15484, 7808, 1667, 9863, 21128, 18568, 4747, 142, 428688, 145, 2194, 658, 148, 2200, 1689, 4248, 1692, 8860, 24734, 3755, 4791, 2744, 184, 30393, 6843, 91324, 17083, 30394, 13510, 9928, 5321, 37076, 213, 37078, 1238, 48347, 23260, 8924, 8926, 1761, 48355, 1767, 3816, 5865, 27379, 2805, 17653, 4856, 763, 32512, 6919, 5897, 20235, 11545, 2331, 54555, 18206, 16159, 143650, 2339, 807, 20264, 2345, 4912, 1845, 15669, 1856, 15683, 23368, 333, 6479, 6480, 337, 13138, 5462, 130398, 3946, 34172, 102788, 3478, 143255, 34202, 1435, 66